In [1]:
# The 'pandas' module is for reading and processing csv files
import pandas as pd

### Data Import
In this document, all datasets are imported and merged together to perform analysis. Simple preprocessing and cleaning for each data files are performed:

In [2]:
state_city_ores = pd.read_csv("us_cities_by_state_SEPT.2023_ores.csv")

# drop the used column 'url'
state_city_ores = state_city_ores.drop(columns=['url'])

# clean the column 'state' as some of them has '_' in between blank spaces,
# and unnecessary '()' after the state
state_city_ores["state"] = state_city_ores["state"].str.replace('_', ' ')
state_city_ores["state"] = state_city_ores["state"].str.replace(r'\s*\([^)]*\)$', '', regex=True)

# rename the columns for future merge
state_city_ores.columns = ['state', 'article_title', 'revision_id', 'article_quality']

# change the data type of 'revision_id' to string to prevent
# scientific notation modification to the long integer
state_city_ores['revision_id'] = state_city_ores['revision_id'].astype('str')

print(state_city_ores.shape)
state_city_ores.head()

(21525, 4)


,state,article_title,revision_id,article_quality
0,Alabama,"Abbeville, Alabama",1171163550,C
1,Alabama,"Adamsville, Alabama",1177621427,C
2,Alabama,"Addison, Alabama",1168359898,C
3,Alabama,"Akron, Alabama",1165909508,GA
4,Alabama,"Alabaster, Alabama",1179139816,C


In [3]:
region_division = pd.read_excel("US States by Region - US Census Bureau.xlsx")

# fill out blank spaces with last valid observation forward to next valid
# for the columns 'REGION' and 'DIVISION'
region_division['REGION'] = region_division['REGION'].fillna(method='ffill')
region_division['DIVISION'] = region_division['DIVISION'].fillna(method='ffill')

# drop empty rows that has null value in the column 'STATE'
region_division = region_division.dropna(subset=['STATE']).reset_index(drop=True)

# drop the unused column 'REGION'
region_division = region_division.drop(['REGION'], axis=1)

# rename the columns for future merge
region_division = region_division.rename(columns={'STATE': 'state', 'DIVISION': 'regional_division'})
print(region_division.shape)
region_division.head()

(50, 2)


,regional_division,state
0,New England,Connecticut
1,New England,Maine
2,New England,Massachusetts
3,New England,New Hampshire
4,New England,Rhode Island


In [4]:
population = pd.read_csv("NST-EST2022-ALLDATA.csv")

# select useful columns from the big datasest
population = population[['REGION', 'DIVISION', 'STATE', 
                         'NAME', 'POPESTIMATE2022']]

# drop rows that are not actual state
population = population[population['STATE'] != 0]

# drop unused columns 'REGION', 'DIVISION', 'STATE'
population = population.drop(columns=['REGION', 'DIVISION', 'STATE'])

# rename the columns for future merge
population.columns = ['state', 'population']

print(population.shape)
population.head()

(52, 2)


,state,population
14,Alabama,5074296
15,Alaska,733583
16,Arizona,7359197
17,Arkansas,3045637
18,California,39029342


### Data Merge
Merge all 3 datasets together, here we have a complete dataframe with all information required for the analysis.

In [5]:
df = pd.merge(state_city_ores, region_division, on="state")
df = pd.merge(df, population, on="state")
print(df.shape)
df

(21525, 6)


,state,article_title,revision_id,article_quality,regional_division,population
0,Alabama,"Abbeville, Alabama",1171163550,C,East South Central,5074296
1,Alabama,"Adamsville, Alabama",1177621427,C,East South Central,5074296
2,Alabama,"Addison, Alabama",1168359898,C,East South Central,5074296
3,Alabama,"Akron, Alabama",1165909508,GA,East South Central,5074296
4,Alabama,"Alabaster, Alabama",1179139816,C,East South Central,5074296
...,...,...,...,...,...,...
21520,Wyoming,"Wamsutter, Wyoming",1169591845,GA,Mountain,581381
21521,Wyoming,"Wheatland, Wyoming",1176370621,GA,Mountain,581381
21522,Wyoming,"Worland, Wyoming",1166347917,GA,Mountain,581381
21523,Wyoming,"Wright, Wyoming",1166334449,GA,Mountain,581381


Save this dataframe as a .csv file for reuse:

In [6]:
df.to_csv("wp_scored_city_articles_by_state.csv", index=False)

Here since there's only 48 states in the complete dataframe, I took a look at the difference and found out that in the list of articles, there are 4 states/areas missing: Nebraska, DC, Puerto Rico, and Connecticut.

In [8]:
len(df['state'].unique())

48

In [7]:
list(set(population['state'].unique()) - set(df['state'].unique()))

['Nebraska', 'District of Columbia', 'Puerto Rico', 'Connecticut']

### Analysis

To obtain the results table in the next section, some preprocessing and calculation is needed. The following dataframe calculates a state-level count of articles, high quality count of articles, and the ratio in ‰.

In [10]:
# total articles per population
count_by_state = df.groupby(['regional_division','state']).size().reset_index(name='total_count')
count_by_state = pd.merge(population, count_by_state, on="state")
count_by_state['total_ratio(‰)'] = 1000 * count_by_state['total_count'] / count_by_state['population']

# high quality articles per population
high_quality = df.copy()
high_quality = high_quality[(high_quality['article_quality']=="FA") | (high_quality['article_quality']=="GA")]

quality_by_state = high_quality.groupby(['regional_division','state']).size().reset_index(name='hq_count')
quality_by_state = pd.merge(quality_by_state, population, on="state")
quality_by_state['hq_ratio(‰)'] = 1000 * quality_by_state['hq_count'] / quality_by_state['population']

per_capita = pd.merge(count_by_state, quality_by_state.drop(['population', 'regional_division'], axis=1), on="state")
cols = list(per_capita)
cols.insert(0, cols.pop(cols.index('regional_division')))
per_capita = per_capita.loc[:, cols]
per_capita.head()

,regional_division,state,population,total_count,total_ratio(‰),hq_count,hq_ratio(‰)
0,East South Central,Alabama,5074296,461,0.090850,53,0.010445
1,Pacific,Alaska,733583,149,0.203113,31,0.042258
2,Mountain,Arizona,7359197,91,0.012365,24,0.003261
3,West South Central,Arkansas,3045637,500,0.164169,72,0.023640
4,Pacific,California,39029342,482,0.012350,172,0.004407


The following dataframe calculates a division-level count of articles, high quality count of articles, and the ratio in ‰.

In [17]:
population_div = pd.read_csv("NST-EST2022-ALLDATA.csv")
population_div = population_div[['REGION', 'DIVISION', 'STATE',
                                 'NAME', 'POPESTIMATE2022']]
population_div = population_div[(population_div['STATE'] == 0) & (population_div['DIVISION'] != '0')]
population_div = population_div.drop(['REGION', 'DIVISION', 'STATE'], axis=1)
population_div.columns = ['regional_division', 'population']

region_total_count = per_capita.groupby('regional_division')['total_count'].sum().reset_index()
region_hq_count = per_capita.groupby('regional_division')['hq_count'].sum().reset_index()

div_per_capita = pd.merge(population_div, region_total_count, on="regional_division")
div_per_capita = pd.merge(div_per_capita, region_hq_count, on="regional_division")
div_per_capita['total_ratio(‰)'] = 1000 * div_per_capita['total_count'] / div_per_capita['population']
div_per_capita['hq_ratio(‰)'] = 1000 * div_per_capita['hq_count'] / div_per_capita['population']

div_per_capita

,regional_division,population,total_count,hq_count,total_ratio(‰),hq_ratio(‰)
0,New England,15129548,1437,225,0.094980,0.014872
1,Middle Atlantic,41910858,3781,1056,0.090215,0.025196
2,East North Central,47097779,4754,714,0.100939,0.015160
3,West North Central,21689816,3578,640,0.164962,0.029507
4,South Atlantic,67452940,1850,525,0.027427,0.007783
5,East South Central,19578002,1529,317,0.078098,0.016192
6,West South Central,41685250,2103,634,0.050449,0.015209
7,Mountain,25514320,1189,336,0.046601,0.013169
8,Pacific,53229044,1304,489,0.024498,0.009187


### Results

In [11]:
# 1. Top 10 US states by coverage
r1 = per_capita.sort_values(by='total_ratio(‰)', ascending=False)[['state', 'total_ratio(‰)', 'population', 'total_count']].head(10)
r1.reset_index(drop=True)

,state,total_ratio(‰),population,total_count
0,Vermont,0.508450,647064,329
1,North Dakota,0.456843,779261,356
2,Maine,0.348651,1385340,483
3,South Dakota,0.341824,909824,311
4,Iowa,0.325885,3200517,1043
5,Alaska,0.203113,733583,149
6,Pennsylvania,0.197040,12972008,2556
7,Michigan,0.176697,10034113,1773
8,Wyoming,0.170284,581381,99
9,New Hampshire,0.167714,1395231,234


In [12]:
# 2. Buttom 10 US states by coverage
r2 = per_capita.sort_values(by='total_ratio(‰)')[['state', 'total_ratio(‰)', 'population', 'total_count']].head(10)
r2.reset_index(drop=True)

,state,total_ratio(‰),population,total_count
0,North Carolina,0.004673,10698973,50
1,Nevada,0.005979,3177772,19
2,California,0.012350,39029342,482
3,Arizona,0.012365,7359197,91
4,Virginia,0.015316,8683619,133
5,Florida,0.018521,22244823,412
6,Oklahoma,0.018658,4019800,75
7,Kansas,0.021449,2937150,63
8,Maryland,0.025468,6164660,157
9,Wisconsin,0.032584,5892539,192


In [13]:
# 3. Top 10 US states by high quality
r3 = per_capita.sort_values(by='hq_ratio(‰)', ascending=False)[['state', 'hq_ratio(‰)', 'population', 'hq_count']].head(10)
r3.reset_index(drop=True)

,state,hq_ratio(‰),population,hq_count
0,Vermont,0.069545,647064,45
1,Wyoming,0.067082,581381,39
2,South Dakota,0.061550,909824,56
3,West Virginia,0.059150,1775156,105
4,Montana,0.048982,1122867,55
5,New Hampshire,0.045154,1395231,63
6,Pennsylvania,0.043632,12972008,566
7,Missouri,0.042571,6177957,263
8,Alaska,0.042258,733583,31
9,New Jersey,0.040921,9261699,379


In [15]:
# 4. Buttom 10 US states by high quality
r4 = per_capita.sort_values(by='hq_ratio(‰)')[['state', 'hq_ratio(‰)', 'population', 'hq_count']].head(10)
r4.reset_index(drop=True)

,state,hq_ratio(‰),population,hq_count
0,North Carolina,0.001869,10698973,20
1,Virginia,0.002073,8683619,18
2,Nevada,0.002517,3177772,8
3,Arizona,0.003261,7359197,24
4,California,0.004407,39029342,172
5,Florida,0.005350,22244823,119
6,New York,0.005641,19677151,111
7,Maryland,0.006813,6164660,42
8,Kansas,0.007490,2937150,22
9,Oklahoma,0.007712,4019800,31


In [14]:
top10_all_states = list(r1.state)
top10_hq_states = list(r3.state)
list(set(top10_all_states) & set(top10_hq_states))

['Wyoming',
 'New Hampshire',
 'Alaska',
 'Pennsylvania',
 'South Dakota',
 'Vermont']

In [16]:
bot10_all_states = list(r2.state)
bot10_hq_states = list(r4.state)
list(set(bot10_all_states) & set(bot10_hq_states))

['Florida',
 'Nevada',
 'Kansas',
 'Oklahoma',
 'Arizona',
 'California',
 'Maryland',
 'North Carolina',
 'Virginia']

In [18]:
# 5. Census divisions by total coverage
r5 = div_per_capita.sort_values(by='total_ratio(‰)', ascending=False)[['regional_division', 'total_ratio(‰)']].head(10)
r5.reset_index(drop=True)

,regional_division,total_ratio(‰)
0,West North Central,0.164962
1,East North Central,0.100939
2,New England,0.094980
3,Middle Atlantic,0.090215
4,East South Central,0.078098
5,West South Central,0.050449
6,Mountain,0.046601
7,South Atlantic,0.027427
8,Pacific,0.024498


In [19]:
# 6. Census divisions by high quality
r6 = div_per_capita.sort_values(by='hq_ratio(‰)', ascending=False)[['regional_division', 'hq_ratio(‰)']].head(10)
r6.reset_index(drop=True)

,regional_division,hq_ratio(‰)
0,West North Central,0.029507
1,Middle Atlantic,0.025196
2,East South Central,0.016192
3,West South Central,0.015209
4,East North Central,0.015160
5,New England,0.014872
6,Mountain,0.013169
7,Pacific,0.009187
8,South Atlantic,0.007783
